In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import MLE
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
import optuna
import numpy as np
import time
# 读取数据
file_path = 'data.xlsx'
data = pd.read_excel(file_path)

# 提取输入和输出
x = data.iloc[:, 1:14]  
y = data.iloc[:, 14]    

# 训练集（80%）和测试集（20%）
x_train_full, x_test, y_train_full, y_test = train_test_split(x, y, test_size=0.2, random_state=2892)

# 划分新的训练集和验证集
train_split_idx = int(0.75 * len(x_train_full)) 
x_train = x_train_full.iloc[:train_split_idx] 
x_val = x_train_full.iloc[train_split_idx:]       
y_train = y_train_full.iloc[:train_split_idx]
y_val = y_train_full.iloc[train_split_idx:] 
# 保存划分后的数据
x_train.to_csv('x_train.csv', index=False)
x_val.to_csv('x_val.csv', index=False)
x_test.to_csv('x_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_val.to_csv('y_val.csv', index=False)
y_test.to_csv('y_test.csv', index=False)


In [ ]:
# 定义目标函数
start_time = time.time()  # 开始时间

def objective(trial):   
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 5, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'minibatch_frac': trial.suggest_float('minibatch_frac', 0.5, 1.0),
        'col_sample': trial.suggest_float('col_sample', 0.5, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 50),
    }
    
    model = NGBRegressor(
        Dist=Normal, 
        Score=MLE,
        natural_gradient=True,
        verbose=False,
        Base=DecisionTreeRegressor(max_depth=param['max_depth']),
        n_estimators=param['n_estimators'],
        learning_rate=param['learning_rate'],
        minibatch_frac=param['minibatch_frac'],
        col_sample=param['col_sample']
    )
    
    model.fit(x_train, y_train)
    y_val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, y_val_pred)  # 使用验证集的 MSE 作为目标函数
    return mse

# 使用Optuna进行超参数优化
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

duration = time.time() - start_time  # 计算运行时间
print(f'Trial completed in {duration:.2f} seconds')  # 打印运行时间

# 获取最佳超参数
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

# 保存最佳超参数
with open('NGB_best_params_with_seeds.txt', 'w') as f:
    f.write(str(best_params))

# 获取并保存Optuna优化过程的详细信息
optuna_results = study.trials_dataframe()
optuna_results.to_csv('NGB_optuna_results_with_seeds.csv', index=False)

# 打印并保存超参数的重要性
param_importances = optuna.importance.get_param_importances(study)
param_importances_df = pd.DataFrame(list(param_importances.items()), columns=['Hyperparameter', 'Importance'])
param_importances_df.to_csv('NGB_param_importances_with_seeds.csv', index=False)
print(param_importances_df)

best_max_depth=best_params.pop('max_depth')
print(best_max_depth)

# 使用最佳超参数训练模型
best_model = NGBRegressor(
    Dist=Normal, 
    Score=MLE,
    natural_gradient=True,
    verbose=True,
    Base=DecisionTreeRegressor(max_depth=best_max_depth),
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    minibatch_frac=best_params['minibatch_frac'],
    col_sample=best_params['col_sample']
)

NGB_model_1 = best_model.fit(x_train, y_train)

# 在训练集上进行预测并计算评估指标
y_train_pred = NGB_model_1.predict(x_train)
r2_train = r2_score(y_train, y_train_pred)
rmse_train = mean_squared_error(y_train, y_train_pred, squared=False)
mae_train = mean_absolute_error(y_train, y_train_pred)
medae_train = median_absolute_error(y_train, y_train_pred)
# 在验证集上进行预测并计算评估指标
y_val_pred = NGB_model_1.predict(x_val)
r2_val = r2_score(y_val, y_val_pred)
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
mae_val = mean_absolute_error(y_val, y_val_pred)
medae_val = median_absolute_error(y_val, y_val_pred)
# 在测试集上进行预测并计算评估指标
best_model_2 = NGBRegressor(
    Dist=Normal, 
    Score=MLE,
    natural_gradient=True,
    verbose=True,
    Base=DecisionTreeRegressor(max_depth=best_max_depth),
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    minibatch_frac=best_params['minibatch_frac'],
    col_sample=best_params['col_sample']
)
NGB_model_2 = best_model_2.fit(x_train_full, y_train_full)
y_test_pred = NGB_model_2.predict(x_test)
r2_test = r2_score(y_test, y_test_pred)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
mae_test = mean_absolute_error(y_test, y_test_pred)
medae_test = median_absolute_error(y_test, y_test_pred)
# 打印并保存评估指标

evaluation_metrics = {
    'Train Set R²': r2_train,
    'Train Set RMSE': rmse_train,
    'Train Set MAE': mae_train,
    'Train Set MedAE': medae_train,
    'Validation Set R²': r2_val,
    'Validation Set RMSE': rmse_val,
    'Validation Set MAE': mae_val,
    'Validation Set MedAE': medae_val,
    'Test Set R²': r2_test,
    'Test Set RMSE': rmse_test,
    'Test Set MAE': mae_test,
    'Test Set MedAE': medae_test,
}
evaluation_metrics_df = pd.DataFrame(list(evaluation_metrics.items()), columns=['Metric', 'Value'])
evaluation_metrics_df.to_csv('NGB_evaluation_metrics.csv', index=False)
print(evaluation_metrics_df)

In [ ]:
import shap
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['Times New Roman']

# 设置随机种子
np.random.seed(0)

# 计算SHAP值
explainer = shap.TreeExplainer(NGB_model_2)
shap_values = explainer.shap_values(x_test)
expected_value = explainer.expected_value

# 创建一个数据框来保存SHAP值
shap_values_df = pd.DataFrame(shap_values, columns=x_test.columns)
shap_values_df['Prediction'] = NGB_model_2.predict(x_test)
shap_values_df['True Value'] = y_test.values
shap_values_df.to_csv('shap_values.csv', index=False)
